In [ ]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

Training with config: {'__module__': 'config', 'BATCH_SIZE': 1280, 'EXP_NAME': 'EXP_03', 'LOSS_FUNC': <class 'icecube.models.LogCoshLoss'>, 'COLLAT_FN': <function collate_fn_v1 at 0x7ffb5739d160>, 'TRN_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV1'>, 'VAL_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV1'>, 'MODEL_NAME': <class 'icecube.models.IceCubeModelEncoderSensorEmbeddinng'>, '__doc__': None}
epoch     train_loss  valid_loss  val_metric
/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(
